In [5]:
from mypytable import MyPyTable 

All columns prior to cleaning:

"SITE_ID","DATE_TIME","TEMPERATURE","TEMPERATURE_DELTA","RELATIVE_HUMIDITY","SOLAR_RADIATION","OZONE",
"PRECIPITATION","WINDSPEED","WIND_DIRECTION",
"SIGMA_THETA","FLOW_RATE","WINDSPEED_SCALAR","WETNESS","SHELTER_TEMPERATURE","QA_CODE","UPDATE_DATE"

To remove:
* "SITE_ID"
* "TEMPERATURE_DELTA"
* "SIGMA_THETA"
* "QA_CODE"
* "UPDATE_DATE"
* "WETNESS"

In [6]:
table = MyPyTable()
table.load_from_file("meteorological-hourly.csv")
clean_table = table.get_multiple_columns(["DATE_TIME","TEMPERATURE","RELATIVE_HUMIDITY","SOLAR_RADIATION","OZONE",\
    "PRECIPITATION","WINDSPEED","WIND_DIRECTION","FLOW_RATE","WINDSPEED_SCALAR","SHELTER_TEMPERATURE"], "", False)
print(clean_table.get_shape())
clean_table.save_to_file("meteorological-hourly-clean.csv")

(36699, 11)
